In [2]:
import pandas as pd
import pickle
import pprint
import random

In [3]:
%%time

individual_provider_payment_df_clean = pd.read_pickle("../data/individual_provider_payment.pkl")

Wall time: 13.3 s


In [4]:
individual_provider_payment_df_clean.describe()

,National Provider Identifier,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Medicare Payment Amount
count,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06
mean,1.499785e+09,1.983183e+02,7.084164e+01,1.151297e+02,9.735926e+01,7.392625e+01
std,2.877289e+08,2.351712e+03,1.442648e+02,2.783331e+02,2.186003e+02,1.705221e+02
min,1.003000e+09,2.400000e+00,1.100000e+01,1.100000e+01,6.035380e-05,0.000000e+00
25%,1.255302e+09,2.000000e+01,1.700000e+01,2.000000e+01,2.501000e+01,1.967050e+01
50%,1.497969e+09,4.300000e+01,3.200000e+01,4.000000e+01,6.639000e+01,4.745194e+01
75%,1.740681e+09,1.160000e+02,7.300000e+01,1.040000e+02,1.131400e+02,8.456263e+01
max,1.993000e+09,3.301134e+06,9.532700e+04,1.758700e+05,3.986417e+04,3.125351e+04


In [5]:
individual_provider_payment_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9416125 entries, 0 to 9416124
Data columns (total 21 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   First Name of the Provider                                object 
 3   Credentials of the Provider                               object 
 4   Entity Type of the Provider                               object 
 5   Street Address 1 of the Provider                          object 
 6   City of the Provider                                      object 
 7   Zip Code of the Provider                                  object 
 8   State Code of the Provider                                object 
 9   Country Code of the Provider                              object 
 10  Provider Type                 

In [5]:
individual_provider_payment_df_clean['Provider Type'].value_counts()

Diagnostic Radiology                   1241400
Internal Medicine                      1118171
Family Practice                         969268
Nurse Practitioner                      560219
Cardiology                              445088
                                        ...   
Ambulance Service Provider                  42
Unknown Supplier/Provider Specialty         17
All Other Suppliers                         11
Medical Toxicology                           2
Slide Preparation Facility                   2
Name: Provider Type, Length: 88, dtype: int64

In [6]:
individual_provider_payment_df_clean['HCPCS Description'].value_counts()

Established patient office or other outpatient visit, typically 15 minutes    457835
Established patient office or other outpatient, visit typically 25 minutes    428262
New patient office or other outpatient visit, typically 45 minutes            179569
New patient office or other outpatient visit, typically 30 minutes            178057
Subsequent hospital inpatient care, typically 25 minutes per day              175335
                                                                               ...  
Drainage of abscess or blood accumulation at forearm and/or wrist                  1
Repair of windpipe cartilage by insertion of splint or graft                       1
Removal of nerve of foot muscle                                                    1
Bone graft of pelvic bone with microvascular connection                            1
Removal of bladder, urinary ducts (ureters)                                        1
Name: HCPCS Description, Length: 5171, dtype: int64

In [7]:
individual_provider_payment_df_clean['HCPCS Code'].value_counts()

99213    457835
99214    428262
99204    179569
99203    178057
99232    175335
          ...  
67120         1
80190         1
44314         1
96420         1
86622         1
Name: HCPCS Code, Length: 5702, dtype: int64

In [8]:
# informational to make sure my row drops will be dropping what I expect it to drop
individual_provider_payment_df_clean.loc[individual_provider_payment_df_clean['HCPCS Description'].str.contains('patient office')]['HCPCS Description'].value_counts()

Established patient office or other outpatient visit, typically 15 minutes    457835
Established patient office or other outpatient, visit typically 25 minutes    428262
New patient office or other outpatient visit, typically 45 minutes            179569
New patient office or other outpatient visit, typically 30 minutes            178057
Established patient office or other outpatient visit, typically 10 minutes    133176
Established patient office or other outpatient, visit typically 40 minutes    119757
New patient office or other outpatient visit, typically 60 minutes             62427
New patient office or other outpatient visit, typically 20 minutes             48225
Established patient office or other outpatient visit, typically 5 minutes      30699
New patient office or other outpatient visit, typically 10 minutes              4619
Name: HCPCS Description, dtype: int64

In [9]:
# remove patient office visits
individual_provider_payment_df_clean_no_office = individual_provider_payment_df_clean.loc[~individual_provider_payment_df_clean['HCPCS Description'].str.contains('patient office')]

In [10]:
# informational to make sure my row drops will be dropping what I expect it to drop
individual_provider_payment_df_clean.loc[individual_provider_payment_df_clean['HCPCS Description'].str.contains('hospital inpatient care,')]['HCPCS Description'].value_counts()

Subsequent hospital inpatient care, typically 25 minutes per day    175335
Initial hospital inpatient care, typically 70 minutes per day       133367
Subsequent hospital inpatient care, typically 35 minutes per day    120664
Initial hospital inpatient care, typically 50 minutes per day       111446
Subsequent hospital inpatient care, typically 15 minutes per day     69468
Initial hospital inpatient care, typically 30 minutes per day        41781
Name: HCPCS Description, dtype: int64

In [11]:
# remove hospital inpatient care visits
individual_provider_payment_df_clean_no_office = individual_provider_payment_df_clean_no_office.loc[~individual_provider_payment_df_clean_no_office['HCPCS Description'].str.contains('hospital inpatient care,')]

In [33]:
individual_provider_payment_df_clean_no_office.head()

,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Credentials of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Country Code of the Provider,...,Place of Service,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Medicare Payment Amount,Total Revenue
0,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,F,99217,Hospital observation care discharge,N,100.0,96,100,73.398800,56.827200,7339.880000
1,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,F,99218,Hospital observation care typically 30 minutes,N,26.0,25,26,100.080000,78.460000,2602.080000
2,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,F,99219,Hospital observation care typically 50 minutes,N,52.0,51,52,136.380000,102.807692,7091.760000
3,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,F,99220,Hospital observation care typically 70 minutes...,N,59.0,59,59,190.363729,141.293559,11231.460000
9,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,F,99239,"Hospital discharge day management, more than 3...",N,341.0,291,341,108.363284,84.056158,36951.880001


In [34]:
individual_provider_payment_df_clean_no_office.to_csv("provider_payments_no_office_hospital_visits.csv", index=False)

In [12]:
individual_provider_payment_df_clean_no_office['HCPCS Description'].value_counts()


Administration of influenza virus vaccine                                                                                                                                                                                                                           104505
Insertion of needle into vein for collection of blood sample                                                                                                                                                                                                         95712
Routine EKG using at least 12 leads including interpretation and report                                                                                                                                                                                              84250
Annual wellness visit, includes a personalized prevention plan of service (pps), subsequent visit                                                                                                      

In [13]:
# creating total revenue column

individual_provider_payment_df_clean_no_office['Total Revenue'] = individual_provider_payment_df_clean_no_office['Average Medicare Allowed Amount'] * individual_provider_payment_df_clean_no_office['Number of Services']

In [14]:
individual_provider_payment_df_clean_no_office['HCPCS Code'].value_counts()

G0008    104505
36415     95712
93000     84250
G0439     76370
96372     70312
          ...  
64630         1
87551         1
42999         1
23030         1
32851         1
Name: HCPCS Code, Length: 5686, dtype: int64

In [15]:
individual_provider_payment_df_clean_no_office['HCPCS Description'].value_counts()

Administration of influenza virus vaccine                                                                                                                                                                                                                           104505
Insertion of needle into vein for collection of blood sample                                                                                                                                                                                                         95712
Routine EKG using at least 12 leads including interpretation and report                                                                                                                                                                                              84250
Annual wellness visit, includes a personalized prevention plan of service (pps), subsequent visit                                                                                                      

In [16]:
individual_provider_payment_df_clean_no_office.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7121438 entries, 0 to 9416124
Data columns (total 22 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   First Name of the Provider                                object 
 3   Credentials of the Provider                               object 
 4   Entity Type of the Provider                               object 
 5   Street Address 1 of the Provider                          object 
 6   City of the Provider                                      object 
 7   Zip Code of the Provider                                  object 
 8   State Code of the Provider                                object 
 9   Country Code of the Provider                              object 
 10  Provider Type                 

In [17]:
# pivot table for viewing

provider_no_office = pd.pivot_table(individual_provider_payment_df_clean_no_office,index=["Provider Type", "HCPCS Code", "HCPCS Description"])

In [18]:
provider_no_office

Average Medicare Allowed Amount  \
Provider Type      HCPCS Code HCPCS Description                                                                     
Addiction Medicine 36415      Insertion of needle into vein for collection of...                         3.000000   
                   36556      Insertion of central venous catheter for infusi...                       112.510000   
                   36620      Insertion of arterial catheter for blood sampli...                        47.990000   
                   70100      X-ray of mandible, less than 4 views                                      12.624219   
                   70310      X-ray of teeth, less than full mouth                                      12.237704   
...                                                                                                           ...   
Vascular Surgery   Q4131      Epifix or epicord, per square centimeter                                 161.975351   
                   Q4148      Neox 1k, per square centimeter                                           250.330000   
                   Q9965      Low osmolar contrast material, 100-199 mg/ml io...                         0.834526   
                   Q9966      Low osmolar contrast material, 200-299 mg/ml io...                         0.246482   
                   Q9967      Low osmolar contrast material, 300-399 mg/ml io...                         0.124234   

                                                                                  Average Medicare Payment Amount  \
Provider Type      HCPCS Code HCPCS Description                                                                     
Addiction Medicine 36415      Insertion of needle into vein for collection of...                         2.933529   
                   36556      Insertion of central venous catheter for infusi...                        88.210000   
                   36620      Insertion of arterial catheter for blood sampli...                        37.620000   
                   70100      X-ray of mandible, less than 4 views                                       9.532713   
                   70310      X-ray of teeth, less than full mouth                                       9.257998   
...                                                                                                           ...   
Vascular Surgery   Q4131      Epifix or epicord, per square centimeter                                 126.948602   
                   Q4148      Neox 1k, per square centimeter                                           196.258910   
                   Q9965      Low osmolar contrast material, 100-199 mg/ml io...                         0.654269   
                   Q9966      Low osmolar contrast material, 200-299 mg/ml io...                         0.192448   
                   Q9967      Low osmolar contrast material, 300-399 mg/ml io...                         0.096476   

                                                                                  National Provider Identifier  \
Provider Type      HCPCS Code HCPCS Description                                                                  
Addiction Medicine 36415      Insertion of needle into vein for collection of...                  1.443171e+09   
                   36556      Insertion of central venous catheter for infusi...                  1.770876e+09   
                   36620      Insertion of arterial catheter for blood sampli...                  1.770876e+09   
                   70100      X-ray of mandible, less than 4 views                                1.306066e+09   
                   70310      X-ray of teeth, less than full mouth                                1.306066e+09   
...                                                                                                        ...   
Vascular Surgery   Q4131      Epifix or epicord, per square centimeter                            1.543785e+09   
            

In [19]:
# sorting by average allowed medicare amount

provider_no_office_sort_payment = individual_provider_payment_df_clean_no_office.sort_values(["Provider Type","Average Medicare Allowed Amount"], ascending=False)

In [20]:
# sorting by total revenue

provider_no_office_sort_revenue = individual_provider_payment_df_clean_no_office.sort_values(["Provider Type","Total Revenue"], ascending=False)

In [21]:
# provider type top 5 by average allowed amount


# create an empty dictionary to store the HCPCS values
provider_no_office_payment_dict = {}

# iterate through the rows
for i, v in provider_no_office_sort_payment.iterrows():
    
    # checking to see if Provider Type is already in the dictionary
    if v['Provider Type'] not in provider_no_office_payment_dict.keys():
        
        # if provider type is not is not already in the dictionary then add that provider type as a key with the HCPCS code as the value
        provider_no_office_payment_dict[v['Provider Type']]=[v['HCPCS Code']]
        
    # if provider type is already in the dictionary, check that the number of HCPCS codes is less than 5; if so, add the HCPCS code
    elif (len(provider_no_office_payment_dict[v['Provider Type']]) < 5) & (v['HCPCS Code'] not in provider_no_office_payment_dict[v['Provider Type']]):
        provider_no_office_payment_dict[v['Provider Type']].append(v['HCPCS Code'])
    
    # if count of HCPCS code is 5, move to the next row...
    else:
        continue

print(provider_no_office_payment_dict)
        

{'Vascular Surgery': ['37227', '37231', '37225', '37229', '37226'], 'Urology': ['Q2043', '55873', 'J2505', 'J9225', 'G0339'], 'Unknown Supplier/Provider Specialty': ['99291', '90791', '36556', '92014', '99239'], 'Undefined Physician type': ['50360', '22633', '63081', '33533', '22612'], 'Thoracic Surgery': ['37227', '37225', '37229', '33945', '32854'], 'Surgical Oncology': ['J2505', '19296', '48150', '48153', '47780'], 'Sports Medicine': ['23472', '27130', '27447', '27245', '29827'], 'Speech Language Pathologist': ['92616', '92612', '92523', '31579', '96125'], 'Slide Preparation Facility': ['88312', '88305'], 'Sleep Medicine': ['21080', '33361', '95811', '95810', '21085'], 'Rheumatology': ['J2505', 'J3590', '78815', 'J7327', 'J9310'], 'Registered Dietitian or Nutrition Professional': ['G0108', '97802', 'G0270', '99490', '97803'], 'Radiation Oncology': ['Q2043', '22513', '22514', 'G0339', 'J2505'], 'Pulmonary Disease': ['78815', '32654', '36478', '95811', '31652'], 'Psychologist, Clinica

In [22]:
# facilities top 5 by average allowed amount

facilities_top_five_avg_pmt = provider_no_office_sort_payment[provider_no_office_sort_payment["Place of Service"] == "F"]

facilities_top_five_avg_pmt_dict = {}
for i, v in facilities_top_five_avg_pmt.iterrows():
    if v['Provider Type'] not in facilities_top_five_avg_pmt_dict.keys():
        facilities_top_five_avg_pmt_dict[v['Provider Type']]=[v['HCPCS Code']]
    elif (len(facilities_top_five_avg_pmt_dict[v['Provider Type']]) < 5) & (v['HCPCS Code'] not in facilities_top_five_avg_pmt_dict[v['Provider Type']]):
        facilities_top_five_avg_pmt_dict[v['Provider Type']].append(v['HCPCS Code'])
    else:
        continue

print(facilities_top_five_avg_pmt_dict)

{'Vascular Surgery': ['34848', '33427', '50360', '34847', '33405'], 'Urology': ['51999', '50360', '51596', '51595', '51597'], 'Unknown Supplier/Provider Specialty': ['99291', '90791', '36556', '99239', '90834'], 'Undefined Physician type': ['50360', '22633', '63081', '33533', '22612'], 'Thoracic Surgery': ['33945', '32854', '32853', '33916', '33860'], 'Surgical Oncology': ['48150', '48153', '47780', '47120', '47379'], 'Sports Medicine': ['23472', '27130', '27447', '27245', '29827'], 'Speech Language Pathologist': ['92523', '31579', '92611', '92526', '92597'], 'Sleep Medicine': ['33361', '92928', '93460', '93456', '95951'], 'Rheumatology': ['29870', '20926', '99291', '42405', '99220'], 'Registered Dietitian or Nutrition Professional': ['G0108', '97802', '97803', 'G0270', 'G0109'], 'Radiation Oncology': ['G0340', 'G0339', '77301', '55875', '77435'], 'Pulmonary Disease': ['32654', '37225', '92928', '37226', '31600'], 'Psychologist, Clinical': ['90791', '99306', '96020', '90792', '90837'],

In [23]:
# in office top 5 by average allowed amount

offices_top_five_avg_pmt = provider_no_office_sort_payment[provider_no_office_sort_payment["Place of Service"] == "O"]

offices_top_five_avg_pmt_dict = {}
for i, v in offices_top_five_avg_pmt.iterrows():
    if v['Provider Type'] not in offices_top_five_avg_pmt_dict.keys():
        offices_top_five_avg_pmt_dict[v['Provider Type']]=[v['HCPCS Code']]
    elif (len(offices_top_five_avg_pmt_dict[v['Provider Type']]) < 5) & (v['HCPCS Code'] not in offices_top_five_avg_pmt_dict[v['Provider Type']]):
        offices_top_five_avg_pmt_dict[v['Provider Type']].append(v['HCPCS Code'])
    else:
        continue
        
print(offices_top_five_avg_pmt_dict)

{'Vascular Surgery': ['37227', '37231', '37225', '37229', '37226'], 'Urology': ['Q2043', '55873', 'J2505', 'J9225', 'G0339'], 'Unknown Supplier/Provider Specialty': ['92014'], 'Undefined Physician type': ['78815', '78492', '14301', '14061', '15260'], 'Thoracic Surgery': ['37227', '37225', '37229', '37228', '37241'], 'Surgical Oncology': ['J2505', '19296', '78815', '60500', '19301'], 'Sports Medicine': ['23470', '29827', 'J7327', '29876', '23412'], 'Speech Language Pathologist': ['92616', '92612', '92523', '31579', '96125'], 'Slide Preparation Facility': ['88312', '88305'], 'Sleep Medicine': ['21080', '95811', '95810', '21085', '95827'], 'Rheumatology': ['J2505', 'J3590', '78815', 'J7327', 'J9310'], 'Registered Dietitian or Nutrition Professional': ['G0108', '97802', 'G0270', '99490', '97803'], 'Radiation Oncology': ['Q2043', '22513', '22514', 'G0339', 'J2505'], 'Pulmonary Disease': ['78815', '36478', '95811', '31652', '95810'], 'Psychologist, Clinical': ['70555', '96020', '99350', '993

In [24]:
# provider type top 5 by revenue

provider_no_office_revenue_dict = {}
for i,v in provider_no_office_sort_revenue.iterrows():
    if v['Provider Type'] not in provider_no_office_revenue_dict.keys():
        provider_no_office_revenue_dict[v['Provider Type']]=[v['HCPCS Code']]
    elif (len(provider_no_office_revenue_dict[v['Provider Type']]) < 5) & (v['HCPCS Code'] not in provider_no_office_revenue_dict[v['Provider Type']]):
        provider_no_office_revenue_dict[v['Provider Type']].append(v['HCPCS Code'])
    else:
        continue

print(provider_no_office_revenue_dict)
        

{'Vascular Surgery': ['37229', '37225', '37228', '37227', '37224'], 'Urology': ['Q2043', 'J1212', 'J3490', 'J0897', '27370'], 'Unknown Supplier/Provider Specialty': ['99291', '90834', '99239', '90832', '90791'], 'Undefined Physician type': ['A9500', '93306', '78452', '94004', '36475'], 'Thoracic Surgery': ['37225', '37229', '37227', '36473', '36478'], 'Surgical Oncology': ['G9678', '93990', 'G0204', '17311', 'J2505'], 'Sports Medicine': ['J7321', '27370', '73580', '77002', 'J7324'], 'Speech Language Pathologist': ['92507', '92609', '92526', '97532', '92616'], 'Slide Preparation Facility': ['88305', '88312'], 'Sleep Medicine': ['J2357', '95811', '95810', 'J0585', '99291'], 'Rheumatology': ['J1745', 'J1602', 'J9310', 'J0129', 'J0717'], 'Registered Dietitian or Nutrition Professional': ['G0108', '97803', 'G0270', '97802', 'G0109'], 'Radiation Oncology': ['77523', '77522', 'A9606', '77525', 'G6015'], 'Pulmonary Disease': ['J2357', '99291', '99310', '95811', 'J2182'], 'Psychologist, Clinica

In [25]:
# facilities top 5 by total revenue

facilities_top_five_revenue = provider_no_office_sort_payment[provider_no_office_sort_payment["Place of Service"] == "F"]

facilities_top_five_revenue_dict = {}
for i, v in facilities_top_five_revenue.iterrows():
    if v['Provider Type'] not in facilities_top_five_revenue_dict.keys():
        facilities_top_five_revenue_dict[v['Provider Type']]=[v['HCPCS Code']]
    elif (len(facilities_top_five_revenue_dict[v['Provider Type']]) < 5) & (v['HCPCS Code'] not in facilities_top_five_revenue_dict[v['Provider Type']]):
        facilities_top_five_revenue_dict[v['Provider Type']].append(v['HCPCS Code'])
    else:
        continue

print(facilities_top_five_revenue_dict)

{'Vascular Surgery': ['34848', '33427', '50360', '34847', '33405'], 'Urology': ['51999', '50360', '51596', '51595', '51597'], 'Unknown Supplier/Provider Specialty': ['99291', '90791', '36556', '99239', '90834'], 'Undefined Physician type': ['50360', '22633', '63081', '33533', '22612'], 'Thoracic Surgery': ['33945', '32854', '32853', '33916', '33860'], 'Surgical Oncology': ['48150', '48153', '47780', '47120', '47379'], 'Sports Medicine': ['23472', '27130', '27447', '27245', '29827'], 'Speech Language Pathologist': ['92523', '31579', '92611', '92526', '92597'], 'Sleep Medicine': ['33361', '92928', '93460', '93456', '95951'], 'Rheumatology': ['29870', '20926', '99291', '42405', '99220'], 'Registered Dietitian or Nutrition Professional': ['G0108', '97802', '97803', 'G0270', 'G0109'], 'Radiation Oncology': ['G0340', 'G0339', '77301', '55875', '77435'], 'Pulmonary Disease': ['32654', '37225', '92928', '37226', '31600'], 'Psychologist, Clinical': ['90791', '99306', '96020', '90792', '90837'],

In [26]:
# in office top 5 by total revenue

offices_top_five_revenue = provider_no_office_sort_payment[provider_no_office_sort_payment["Place of Service"] == "O"]

offices_top_five_revenue_dict = {}
for i, v in offices_top_five_revenue.iterrows():
    if v['Provider Type'] not in offices_top_five_revenue_dict.keys():
        offices_top_five_revenue_dict[v['Provider Type']]=[v['HCPCS Code']]
    elif (len(offices_top_five_revenue_dict[v['Provider Type']]) < 5) & (v['HCPCS Code'] not in offices_top_five_revenue_dict[v['Provider Type']]):
        offices_top_five_revenue_dict[v['Provider Type']].append(v['HCPCS Code'])
    else:
        continue
        
print(offices_top_five_revenue_dict)

{'Vascular Surgery': ['37227', '37231', '37225', '37229', '37226'], 'Urology': ['Q2043', '55873', 'J2505', 'J9225', 'G0339'], 'Unknown Supplier/Provider Specialty': ['92014'], 'Undefined Physician type': ['78815', '78492', '14301', '14061', '15260'], 'Thoracic Surgery': ['37227', '37225', '37229', '37228', '37241'], 'Surgical Oncology': ['J2505', '19296', '78815', '60500', '19301'], 'Sports Medicine': ['23470', '29827', 'J7327', '29876', '23412'], 'Speech Language Pathologist': ['92616', '92612', '92523', '31579', '96125'], 'Slide Preparation Facility': ['88312', '88305'], 'Sleep Medicine': ['21080', '95811', '95810', '21085', '95827'], 'Rheumatology': ['J2505', 'J3590', '78815', 'J7327', 'J9310'], 'Registered Dietitian or Nutrition Professional': ['G0108', '97802', 'G0270', '99490', '97803'], 'Radiation Oncology': ['Q2043', '22513', '22514', 'G0339', 'J2505'], 'Pulmonary Disease': ['78815', '36478', '95811', '31652', '95810'], 'Psychologist, Clinical': ['70555', '96020', '99350', '993

In [27]:
len(provider_no_office_payment_dict)



88

In [28]:
#convert to data frame

provider_no_office_by_payment_df = pd.DataFrame.from_dict(provider_no_office_payment_dict, orient='index') 
provider_facilities_by_payment_df = pd.DataFrame.from_dict(facilities_top_five_avg_pmt_dict, orient='index')
provider_offices_by_payment_df = pd.DataFrame.from_dict(offices_top_five_avg_pmt_dict, orient='index')
provider_no_office_by_revenue_df = pd.DataFrame.from_dict(provider_no_office_revenue_dict, orient='index')
provider_facilities_by_revenue_df = pd.DataFrame.from_dict(facilities_top_five_revenue_dict, orient='index')
provider_offices_by_revenue_df = pd.DataFrame.from_dict(offices_top_five_revenue_dict, orient='index')

In [29]:
pd.options.display.max_rows = 100
provider_no_office_by_payment_df

,0,1,2,3,4
Vascular Surgery,37227,37231,37225,37229,37226
Urology,Q2043,55873,J2505,J9225,G0339
Unknown Supplier/Provider Specialty,99291,90791,36556,92014,99239
Undefined Physician type,50360,22633,63081,33533,22612
Thoracic Surgery,37227,37225,37229,33945,32854
Surgical Oncology,J2505,19296,48150,48153,47780
Sports Medicine,23472,27130,27447,27245,29827
Speech Language Pathologist,92616,92612,92523,31579,96125
Slide Preparation Facility,88312,88305,None,None,None
Sleep Medicine,21080,33361,95811,95810,21085


In [30]:
# write to csv for easier read in

provider_no_office_by_payment_df.to_csv("../data/provider_services_o_f_by_payment.csv")
provider_facilities_by_payment_df.to_csv("../data/provider_services_f_by_payment.csv")
provider_offices_by_payment_df.to_csv("../data/provider_services_o_by_payment.csv")
provider_no_office_by_revenue_df.to_csv("../data/provider_services_o_f_by_revenue.csv")
provider_facilities_by_revenue_df.to_csv("../data/provider_services_f_by_payment.csv")
provider_offices_by_revenue_df.to_csv("../data/provider_services_o_by_payment.csv")

In [31]:
names = ['Amanda', 'Ness', 'Cat', 'Ben', 'Jacob', 'Sophia']


In [32]:
generator = names.pop(random.randrange(0,len(names),1))
print(generator)

Amanda
